# 1.1 부모-자식 분할
- 문서의 구조적 정보를 더 잘 활용하기 위한 방법으로 부모-자식분할이 있습니다.
- 문서를 계층적으로 분할하여 원본 문서를 큰 단위의 부모 문서로 나누고, 이를 다시 작은 단위의 자식 문서로 세분화합니다.
- 원본문서 -> 부모문서 -> 자식문서 3단계 구조를 형성합니다.
- 부모-자식 분할 방식은 문서의 저장과 검색에서 이원화된 접근법을 채택합니다.
- 문서의 계층 구조를 유지하면서도 효율적인 검색을 위해 자식 문서는 벡터 데이터베이스에 임베딩하여 저장하고, 부모 문서는 별도의 저장소에 원본 형태로 보관합니다.
- 실제 검색시에는 자식 문서를 기반으로 유사성 검색을 수행하지만, 최종적으로 반환되는 문서는 부모 문서입니다.
- 정확한 정보 : 검색시 자식문서를 기반으로 검색
- 넓은 맥락 : 반환은 부모문서로 하므로 전체적인 맥락까지 함께 파악할 수 있습니다.

## 동작방식
1. 문서 분할
  - 먼저 원본 문서를 비교적 큰 크기의 부모 문서로 나눕니다
  - 이때 문서의 구조적 특성(장, 절, 단락)을 고려할 수 있습니다.
  - 이후 각 부모 문서를 더 작은 자식 문서로 나눕니다.
  - 이 과정에서 의미 기반 분할 같은 다른 기술을 활용할 수 있습니다.

2. 메타데이터 할당
  - 각 자식 문서에 해당 부모 문서의 식별자를 메타데이터로 할당합니다.
  - 자식문서와 부모문서간의 관계를 추적할 수 있습니다.

3. 임베딩 저장
  - 자식 문서는 벡터 데이터베이스에 저장합니다.
  - 각 청크의 텍스트 내용은 임베딩되어 벡터 형태로 저장됩니다.
  - 부모 문서는 별도의 문서 저장소에 저장됩니다.

## 검색과정
1. 사용자 쿼리가 입력되면, 먼저 벡터 데이터베이스에서 쿼리와 가장 유사한 자식 문서를 검색합니다.
2. 검색된 자식 문서의 메타데이터를 확인하여 해당하는 부모 문서의 식별자를 찾습니다.
3. 찾은 식별자를 이용해 문서 저장소에서 관련된 부모 문서를 반환합니다.

- 부모-자식 분할이 의미 기반 분할을 대체하는 것이 아니라, 두 기술을 보완적으로 사용할 수 있다는 점입니다.
- 부모 문서를 자식 문서로 나눌 때 의미 기반 분할을 적용하여 의미적으로 더 일관된 청크를 만들 수도 있습니다.

- 텍스트 파일을 사용하므로 랭체인의 TextLoader 클래스를 사용합니다. 텍스트 파일을 읽어 랭체인의 Document 객체로 변환하는 클래스입니다.

In [5]:
from langchain_community.document_loaders import TextLoader

# 문서 로더 설정
loaders = [
    TextLoader("data/How_to_invest_money.txt", encoding="utf-8"),    
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

- 이제 부모-자식 분할에 필요한 설정을 진행합니다. 재귀적 문자 텍스트 분할 방식을 활용하여 부모 자식 문서를 생성합니다.

In [2]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 부모 문서 생성을 위한 텍스트 분할기
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
# 자식 문서 생성을 위한 텍스트 분할기(부모보다 작은 크기로 설정
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# 자식 문서 인덱싱을 위한 벡터 저장소
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OllamaEmbeddings(model="bge-m3")
)
# 부모 문서 저장을 위한 저장소
store = InMemoryStore()

- 자식 문서 저장소는 Chroma 벡터 데이터베이스를 사용합니다.
- 부모 문서 저장소는 InMemoryStore를 사용합니다.

In [6]:
# ParentDocumentRetriever 설정
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# 문서 추가
retriever.add_documents(docs)

# 부모 문서 수 확인
print(f"Number of parent documents: {len(list(store.yield_keys()))}")

Number of parent documents: 219


- ParentDocumentRetriever
  - 자식 문서 저장소, 부모 문서 저장소, 자식 문서 텍스트 분할기, 부모 문서 텍스트 분할기
- 문서들이 자동으로 부모와 자식 문서로 분할되어 각각의 저장소에 저장됩니다.
- 이후 store.yield_keys()를 사용하여 저장된 모든 부모 문서의 키를 가져온 뒤, 해당 키의 개수를 세어 총 부모 문서의 수를 확인하고 문서 분할 과정과 저장이 제대로 이루어졌는지 검증합니다.

In [8]:
# 질문 정의
query = "What are the types of investments?"

# 연관 문서 수집
retrieved_docs = retriever.invoke(query)

# 첫 번째 연관 문서 출력
print(f"Parent Document: {retrieved_docs[0].page_content}")

Parent Document: There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.

Keeping these five general factors in mind, the present chapter will
discuss real-estate mortgages as a form of investment, both as adapted
to the requirements of private funds and of a business surplus.


- 부모-자식 분할의 작동 방식을 이해하기 위해 벡터 저장소에서 직접 자식 문서를 검색해서 첫 번째 자식 문서를 출력해보겠습니다.

In [9]:
#자식 문서 검색
query = "What are the types of investments?"
sub_docs = vectorstore.similarity_search(query)
print(f"Child Document: {sub_docs[0].page_content}")

Child Document: forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.


- 자식 문서는 "투자의 다섯 가지 주요 고려사항"이라는 질문에 직접적으로 관련된 핵심 정보를 간결하게 제공합니다.
- 부모 문서는 이 정보를 포함하면서도, 부동산 담보 대출에 대해 논의할 것이라는 추가적인 맥락을 제공합니다.